# SuperPandas SQL Accessor Integration Example

This notebook demonstrates how to use the SQL accessor to query pandas DataFrames using SQL syntax, both with and without the SuperDataFrame functionality.

## Setup

First, let's import the necessary libraries:

In [ ]:
import superpandas as spd  # This registers the SQL accessor
import pandas as pd
import numpy as np

## Basic SQL Accessor Examples

Let's start with some basic SQL query examples:

In [ ]:
# Create sample DataFrames
employees = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "name": ["Alice", "Bob", "Charlie", "Diana", "Eve"],
    "age": [25, 30, 35, 28, 32],
    "department": ["Engineering", "Sales", "Engineering", "Marketing", "Sales"],
    "salary": [80000, 70000, 90000, 75000, 72000]
})

departments = pd.DataFrame({
    "dept_id": [1, 2, 3],
    "name": ["Engineering", "Sales", "Marketing"],
    "budget": [500000, 300000, 200000]
})

print("Employees DataFrame:")
display(employees)
print("\nDepartments DataFrame:")
display(departments)

### 1. Basic Query - Employees over 30

In [ ]:
result = employees.sql.query("SELECT * FROM df WHERE age > 30")
print("Employees over 30:")
display(result)

### 2. Aggregation - Average Salary by Department

In [ ]:
result = employees.sql.query("""
    SELECT department, AVG(salary) as avg_salary, COUNT(*) as count
    FROM df 
    GROUP BY department
    ORDER BY avg_salary DESC
""")
print("Average salary by department:")
display(result)

### 3. Join Query - Employees with Department Budgets

In [ ]:
env = {"departments": departments}
result = employees.sql.query("""
    SELECT 
        e.name,
        e.department,
        e.salary,
        d.budget,
        ROUND(e.salary * 100.0 / d.budget, 2) as salary_percent
    FROM df e
    JOIN departments d ON e.department = d.name
    ORDER BY salary_percent DESC
""", env=env)
print("Employees with department budgets:")
display(result)

## SuperDataFrame SQL Examples

Now let's see how SQL queries work with SuperDataFrames that have rich metadata:

In [ ]:
# Create a SuperDataFrame with metadata
sales_data = pd.DataFrame({
    "date": pd.date_range("2023-01-01", periods=10, freq="D"),
    "product": ["A", "B", "A", "C", "B", "A", "C", "B", "A", "C"],
    "quantity": [10, 5, 15, 8, 12, 20, 6, 9, 18, 11],
    "price": [100, 150, 100, 200, 150, 100, 200, 150, 100, 200],
    "region": ["North", "South", "North", "East", "South", "North", "East", "South", "North", "East"]
})

# Convert to SuperDataFrame with metadata
sdf = spd.create_super_dataframe(
    df=sales_data,
    name="daily_sales",
    description="Daily sales data with product, quantity, price, and region information",
    column_descriptions={
        "date": "Date of the sale",
        "product": "Product identifier (A, B, or C)",
        "quantity": "Number of units sold",
        "price": "Price per unit in USD",
        "region": "Sales region (North, South, East, West)"
    }
)

print("SuperDataFrame metadata:")
print(f"Name: {sdf.super.name}")
print(f"Description: {sdf.super.description}")
print(f"Column descriptions: {sdf.super.column_descriptions}")

print("\nSales Data:")
display(sdf)

### 1. Total Revenue by Product

In [ ]:
result = sdf.sql.query("""
    SELECT 
        product,
        SUM(quantity * price) as total_revenue,
        AVG(price) as avg_price,
        COUNT(*) as sales_count
    FROM df 
    GROUP BY product
    ORDER BY total_revenue DESC
""")
print("Total revenue by product:")
display(result)

### 2. Sales Performance by Region

In [ ]:
result = sdf.sql.query("""
    SELECT 
        region,
        SUM(quantity * price) as total_revenue,
        AVG(quantity) as avg_quantity,
        COUNT(*) as transaction_count
    FROM df 
    GROUP BY region
    ORDER BY total_revenue DESC
""")
print("Sales performance by region:")
display(result)

### 3. Daily Sales Trend

In [ ]:
result = sdf.sql.query("""
    SELECT 
        date,
        SUM(quantity * price) as daily_revenue,
        COUNT(*) as transactions
    FROM df 
    GROUP BY date
    ORDER BY date
""")
print("Daily sales trend:")
display(result)

## Complex SQL Examples

Let's explore more complex SQL queries with multiple related tables:

In [ ]:
# Create multiple related DataFrames
orders = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5, 6],
    "customer_id": [101, 102, 101, 103, 102, 104],
    "order_date": pd.date_range("2023-01-01", periods=6, freq="D"),
    "total_amount": [150, 300, 200, 450, 120, 600]
})

customers = pd.DataFrame({
    "customer_id": [101, 102, 103, 104, 105],
    "name": ["John", "Jane", "Bob", "Alice", "Charlie"],
    "email": ["john@email.com", "jane@email.com", "bob@email.com", "alice@email.com", "charlie@email.com"],
    "join_date": pd.date_range("2022-01-01", periods=5, freq="ME")
})

order_items = pd.DataFrame({
    "order_id": [1, 1, 2, 3, 4, 4, 5, 6, 6],
    "product_id": ["P1", "P2", "P1", "P3", "P2", "P4", "P1", "P3", "P4"],
    "quantity": [2, 1, 3, 1, 2, 1, 1, 2, 1],
    "unit_price": [50, 50, 50, 200, 50, 350, 50, 200, 350]
})

products = pd.DataFrame({
    "product_id": ["P1", "P2", "P3", "P4"],
    "name": ["Widget A", "Widget B", "Gadget X", "Gadget Y"],
    "category": ["Widgets", "Widgets", "Gadgets", "Gadgets"],
    "cost": [30, 35, 150, 280]
})

print("Orders DataFrame:")
display(orders)
print("\nCustomers DataFrame:")
display(customers)
print("\nOrder Items DataFrame:")
display(order_items)
print("\nProducts DataFrame:")
display(products)

### Complex Query - Customer Purchase Analysis

In [ ]:
env = {
    "customers": customers,
    "order_items": order_items,
    "products": products
}

result = orders.sql.query("""
    SELECT 
        c.name as customer_name,
        c.email,
        COUNT(DISTINCT o.order_id) as total_orders,
        SUM(o.total_amount) as total_spent,
        AVG(o.total_amount) as avg_order_value,
        STRFTIME('%Y-%m', o.order_date) as order_month,
        GROUP_CONCAT(DISTINCT p.category) as categories_purchased
    FROM df o
    JOIN customers c ON o.customer_id = c.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    GROUP BY c.customer_id, c.name, c.email, STRFTIME('%Y-%m', o.order_date)
    ORDER BY total_spent DESC
""", env=env)

print("Customer purchase analysis:")
display(result)

## Advanced SQL Features

Let's explore some advanced SQL features available in the SQL accessor:

In [ ]:
# Create a sample DataFrame with different data types
sample_df = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "name": ["Alice", "Bob", "Charlie", "Diana", "Eve"],
    "score": [85.5, 92.3, 78.9, 95.1, 88.7],
    "is_active": [True, False, True, True, False],
    "created_date": pd.date_range("2023-01-01", periods=5, freq="D")
})

print("Sample DataFrame:")
display(sample_df)

### String Functions and Pattern Matching

In [ ]:
result = sample_df.sql.query("""
    SELECT 
        name,
        UPPER(name) as upper_name,
        LENGTH(name) as name_length,
        SUBSTR(name, 1, 3) as name_prefix
    FROM df 
    WHERE name LIKE '%a%'
    ORDER BY name_length DESC
""")
print("String functions and pattern matching:")
display(result)

### Date Functions

In [ ]:
result = sample_df.sql.query("""
    SELECT 
        name,
        created_date,
        STRFTIME('%Y-%m', created_date) as year_month,
        STRFTIME('%W', created_date) as week_number,
        STRFTIME('%w', created_date) as day_of_week
    FROM df 
    ORDER BY created_date
""")
print("Date functions:")
display(result)

### Conditional Logic with CASE Statements

In [ ]:
result = sample_df.sql.query("""
    SELECT 
        name,
        score,
        is_active,
        CASE 
            WHEN score >= 90 THEN 'Excellent'
            WHEN score >= 80 THEN 'Good'
            WHEN score >= 70 THEN 'Average'
            ELSE 'Needs Improvement'
        END as grade,
        CASE 
            WHEN is_active = 1 THEN 'Active'
            ELSE 'Inactive'
        END as status
    FROM df 
    ORDER BY score DESC
""")
print("Conditional logic with CASE statements:")
display(result)

## Error Handling Examples

Let's see how the SQL accessor handles various error conditions:

In [ ]:
# Test empty query
try:
    result = sample_df.sql.query("")
except ValueError as e:
    print(f"Empty query error: {e}")

# Test invalid table reference
try:
    result = sample_df.sql.query("SELECT * FROM nonexistent_table")
except RuntimeError as e:
    print(f"Invalid table error: {e}")

# Test invalid env parameter
try:
    result = sample_df.sql.query("SELECT * FROM df", env="not_a_dict")
except TypeError as e:
    print(f"Invalid env type error: {e}")

## Summary

The SQL accessor in SuperPandas provides a powerful way to query pandas DataFrames using familiar SQL syntax. Key features include:

- **In-memory SQLite database** for fast queries
- **Multiple table support** via the `env` parameter
- **Full SQL support** including SELECT, WHERE, JOIN, GROUP BY, etc.
- **Type safety** with proper error handling
- **Custom database URIs** for persistent storage
- **Seamless integration** with existing SuperDataFrame functionality

The SQL accessor is automatically registered when you import SuperPandas, making it available on all pandas DataFrames in your environment.